In [1]:
# file: features_eth_1s.py  (run on Glue/EMR or local Spark with S3 access)
from pyspark.sql import SparkSession, functions as F, Window as W

spark = (SparkSession.builder.appName("eth_features_1s")
         .config("spark.sql.session.timeZone", "UTC")
         .getOrCreate())

start_dt, end_dt = "2024-02-16", "2024-02-23"   # <- adjust / parametrize

VBox()

Starting Spark application


ID,Kind,State,Spark UI,Driver log,User,Current session?
0,pyspark,idle,Link,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
spark.sql("""
    CREATE EXTERNAL TABLE IF NOT EXISTS crypto_data.ethereum_trades (
        price DOUBLE,  -- ← Force DOUBLE here
        -- other columns
    )
    PARTITIONED BY (dt STRING)
    STORED AS PARQUET
    LOCATION 's3://ethereum-kbarczak/cryptolake/raw/trades/'
    TBLPROPERTIES (
        'parquet.enable.dictionary'='true',
        'parquet.compression'='SNAPPY'
    )
""")

spark.sql("MSCK REPAIR TABLE crypto_data.ethereum_trades")

In [9]:
def resample_timeseries(df, cols, interval_us, timestamp_col="ts", group_cols=None, 
                       return_as_timestamp=False, input_unit_us=1):
    """
    Resamples a time-series DataFrame to a specified interval using last-observation-carried-forward (LOCF).
    
    This function creates a dense time grid at the specified interval spanning the input data's 
    time range, then performs forward-fill logic to populate missing timestamps with the last known values.
    
    Works in MICROSECONDS by default (aligns with Spark's native TimestampType precision).
    
    Args:
        df (DataFrame): Source DataFrame with time-series data
        cols (list of str): Column names to resample and forward-fill (value columns)
        interval_us (int): Target resampling interval in MICROSECONDS.
                          Examples:
                          - 1_000_000 = 1 second
                          - 5_000_000 = 5 seconds
                          - 60_000_000 = 1 minute
                          - 100_000 = 100 milliseconds
                          - 1_000 = 1 millisecond
                          - 1 = 1 microsecond
        timestamp_col (str, optional): Name of the timestamp column. Defaults to "ts".
        group_cols (list of str, optional): Grouping/ID columns to preserve (e.g., ["exchange_id", "symbol"]).
                                           If provided, resampling is done separately for each group.
                                           Defaults to None (no grouping).
        return_as_timestamp (bool, optional): If True, converts the output timestamp column to Timestamp dtype.
                                             If False, keeps as LONG in the same unit as input.
                                             Defaults to False.
        input_unit_us (float, optional): Conversion factor from input units to microseconds.
                                        This is the number you MULTIPLY the input by to get microseconds.
                                        Examples:
                                        - 0.001 = input is in nanoseconds (divide by 1000)
                                        - 1 = input is in microseconds (default, no conversion)
                                        - 1_000 = input is in milliseconds (multiply by 1000)
                                        - 1_000_000 = input is in seconds (multiply by 1,000,000)
                                        Defaults to 1 (microseconds).
    
    Returns:
        DataFrame or None: Resampled DataFrame with specified interval resolution and forward-filled values.
                          Returns None if input DataFrame is empty.
    
    Example:
        >>> # Time unit constants (conversion factors to microseconds)
        >>> NANO = 0.001        # nanoseconds: multiply by 0.001 (divide by 1000)
        >>> US = 1              # microseconds: no conversion
        >>> MS = 1_000          # milliseconds: multiply by 1000
        >>> SECOND = 1_000_000  # seconds: multiply by 1,000,000
        >>> 
        >>> # Interval constants (in microseconds)
        >>> INTERVAL_1US = 1
        >>> INTERVAL_1MS = 1_000
        >>> INTERVAL_100MS = 100_000
        >>> INTERVAL_1S = 1_000_000
        >>> INTERVAL_1M = 60_000_000
        
        >>> # Example 1: Input in nanoseconds (high-frequency trading)
        >>> resampled = resample_timeseries(
        ...     nano_df,  # ts column: 1708185600123456789 (nanoseconds)
        ...     ["price", "volume"], 
        ...     interval_us=INTERVAL_1MS,  # 1 millisecond intervals
        ...     input_unit_us=NANO  # 0.001 - converts ns to us
        ... )
        >>> # Output: ts in nanoseconds (converted back)
        
        >>> # Example 2: Input in microseconds (default, best practice)
        >>> resampled = resample_timeseries(
        ...     trades_df,  # ts column: 1708185600123456 (microseconds)
        ...     ["price", "volume"], 
        ...     interval_us=INTERVAL_1S  # 1 second intervals
        ... )
        >>> # Output: ts in microseconds
        
        >>> # Example 3: Input in milliseconds (very common)
        >>> resampled = resample_timeseries(
        ...     trades_df,  # ts column: 1708185600123 (milliseconds)
        ...     ["price", "volume"], 
        ...     interval_us=INTERVAL_1S,
        ...     input_unit_us=MS  # 1000
        ... )
        >>> # Output: ts in milliseconds (converted back)
        
        >>> # Example 4: Input in seconds (Unix timestamps)
        >>> resampled = resample_timeseries(
        ...     trades_df,  # ts column: 1708185600 (seconds)
        ...     ["price", "volume"], 
        ...     interval_us=60 * INTERVAL_1S,  # 1 minute intervals
        ...     input_unit_us=SECOND  # 1_000_000
        ... )
        >>> # Output: ts in seconds (converted back)
        
        >>> # Example 5: Return as Timestamp for human readability
        >>> resampled = resample_timeseries(
        ...     trades_df,
        ...     ["price", "volume"],
        ...     interval_us=INTERVAL_1S,
        ...     input_unit_us=MS,
        ...     return_as_timestamp=True
        ... )
        >>> # Output: ts as TimestampType (2024-02-17 12:00:00)
        
        >>> # Example 6: With grouping (multiple symbols)
        >>> resampled = resample_timeseries(
        ...     trades_df,
        ...     ["price", "volume"],
        ...     interval_us=INTERVAL_100MS,
        ...     input_unit_us=MS,
        ...     group_cols=["symbol", "exchange"]
        ... )
        >>> # Output: ts, symbol, exchange, price, volume
    
    Notes:
        - Uses LOCF (Last Observation Carried Forward) to fill gaps
        - Works in MICROSECONDS internally (matches Spark's TimestampType precision)
        - When group_cols is specified, each group gets its own complete time grid
        - Output timestamps are in the same unit as input (unless return_as_timestamp=True)
        - For nanosecond data: use input_unit_us=0.001 (sub-microsecond precision is lost)
        - Conversion formula: microseconds = input_timestamp * input_unit_us
    """
    if group_cols is None:
        group_cols = []
    
    # Convert input timestamps to microseconds if needed
    if input_unit_us != 1:
        df = df.withColumn(timestamp_col, (F.col(timestamp_col) * input_unit_us).cast("long"))
    
    # Get time bounds (overall or per group)
    if group_cols:
        # Get min/max per group
        bounds = df.groupBy(*group_cols).agg(
            F.min(timestamp_col).alias("min_ts"),
            F.max(timestamp_col).alias("max_ts")
        )
        
        if bounds.count() == 0:  # Empty DataFrame
            return None
        
        # Create bucket boundaries per group
        bounds = bounds.withColumn(
            "min_bucket", (F.col("min_ts") / interval_us).cast("long") * interval_us
        ).withColumn(
            "max_bucket", (F.col("max_ts") / interval_us).cast("long") * interval_us
        )
        
        # Generate grid per group using explode and sequence
        grid = bounds.select(
            *group_cols,
            F.explode(
                F.sequence(
                    F.col("min_bucket"),
                    F.col("max_bucket"),
                    F.lit(interval_us)
                )
            ).alias(timestamp_col)
        )
    else:
        # Get overall time bounds
        mins = df.agg(
            F.min(timestamp_col).alias("min_ts"), 
            F.max(timestamp_col).alias("max_ts")
        ).first()
        
        if mins.min_ts is None:  # Empty DataFrame
            return None
        
        # Calculate bucket boundaries (truncate to interval)
        min_bucket = (mins.min_ts // interval_us) * interval_us
        max_bucket = (mins.max_ts // interval_us) * interval_us
        
        # Generate complete time grid (sequence of buckets)
        grid = spark.range(
            min_bucket, 
            max_bucket + interval_us,  # +interval_us to include the last bucket
            interval_us
        ).select(F.col("id").alias(timestamp_col))
    
    # Downsample source to target interval (keep last value per bucket and group)
    # Create bucket column by dividing timestamp by interval
    group_keys = group_cols + ["bucket"]
    downsampled = df.withColumn("bucket", (F.col(timestamp_col) / interval_us).cast("long") * interval_us) \
        .groupBy(*group_keys) \
        .agg(*[F.last(c, ignorenulls=True).alias(c) for c in cols]) \
        .select(*group_cols, F.col("bucket").alias(timestamp_col), *[F.col(c) for c in cols])
    
    # Join to grid
    join_keys = group_cols + [timestamp_col] if group_cols else [timestamp_col]
    joined = grid.join(downsampled, join_keys, "left")
    
    # Forward-fill missing values using window function
    # Window is partitioned by group_cols (if any) and ordered by timestamp
    if group_cols:
        w = W.partitionBy(*group_cols).orderBy(timestamp_col).rowsBetween(W.unboundedPreceding, 0)
    else:
        w = W.orderBy(timestamp_col).rowsBetween(W.unboundedPreceding, 0)
    
    for c in cols:
        joined = joined.withColumn(c, F.last(c, ignorenulls=True).over(w))
    
    # Convert output timestamps
    if return_as_timestamp:
        # Convert microseconds to Timestamp (perfect alignment!)
        joined = joined.withColumn(
            timestamp_col, 
            (F.col(timestamp_col) / 1_000_000).cast("timestamp")
        )
    elif input_unit_us != 1:
        # Convert back to original input unit
        joined = joined.withColumn(
            timestamp_col,
            (F.col(timestamp_col) / input_unit_us).cast("long")
        )
    
    return joined

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
trades = spark.table("crypto_data.ethereum_trades") \
    .where(F.col("dt").between(F.lit(start_dt), F.lit(end_dt)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
trades.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------+------+-------------------+----+-------------------+----------+
|        id|  price|amount|          timestamp|side|  receipt_timestamp|        dt|
+----------+-------+------+-------------------+----+-------------------+----------+
|1015897581| 2881.2|0.0101|1708300799997999872| buy|1708300800000819712|2024-02-19|
|1015897582| 2881.2| 0.018|1708300799999000064| buy|1708300800008328704|2024-02-19|
|1015897583| 2881.2|0.0156|1708300800000000000| buy|1708300800008368896|2024-02-19|
|1015897584|2881.19|0.0199|1708300800000999936|sell|1708300800008384512|2024-02-19|
|1015897585|2881.19|0.0142|1708300800000999936|sell|1708300800039269888|2024-02-19|
|1015897586| 2881.2|0.0278|1708300800004000000| buy|1708300800039296000|2024-02-19|
|1015897587|2881.19|0.0127|1708300800004000000|sell|1708300800292273152|2024-02-19|
|1015897588| 2881.2| 0.009|1708300800005000192| buy|1708300800292348672|2024-02-19|
|1015897589|2881.19|0.0352|1708300800005000192|sell|1708300800292365824|2024

In [17]:
trades.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

6528794

In [12]:
# Usage examples:
trades_1s = resample_timeseries(df=trades, cols=["price", "amount", "side"], interval_us=1_000_000, timestamp_col="timestamp", return_as_timestamp=True, input_unit_us=1/1_000)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
trades_1s.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

691200

In [18]:
trades_1s.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
An error occurred while calling o391.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 6 in stage 21.0 failed 4 times, most recent failure: Lost task 6.3 in stage 21.0 (TID 105) (ip-172-31-20-64.eu-west-2.compute.internal executor 11): org.apache.spark.SparkException: Parquet column cannot be converted in file s3://ethereum-kbarczak/cryptolake/raw/trades/dt=2024-02-16/converted_fbb71ea129d14e4d96822abbe9739644.snappy.parquet. Column: [price], Expected: double, Found: FLOAT.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.unsupportedSchemaColumnConvertError(QueryExecutionErrors.scala:856)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:442)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.$anonfun$hasNext$1(FileScanRDD.scala:272)
	at scala.runtime.java8.JFunction0$mcZ$sp.apply(JFunction0$mcZ$sp.java:23)
	at org.apache.spark.util.FileAccessContext$.withC